In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

load_dotenv()

True

In [3]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader_multiple_pages = WebBaseLoader(urls)

In [10]:

documents = loader_multiple_pages.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings)

retriever = vectorstore.as_retriever()

In [11]:
retriever.invoke("agent memory")

[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfac

In [17]:
llm = ChatOpenAI(
    model="gpt-4o-mini"
)

In [121]:

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}
Question: {question}

Output in JSON format:
relevance: true or false
""",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = (
        prompt
        | llm
        | JsonOutputParser()
)

In [131]:
question = "agent memory"
docs = vectorstore.similarity_search(question)
result = chain.invoke({"question": question, "context": docs})

answer_prompt = hub.pull("rlm/rag-prompt")
rag_chain = (
        answer_prompt
        | llm
        | StrOutputParser()
)

hallucination_prompt = PromptTemplate(
    template="""
        You are given a context and an answer. Your task is to determine if the answer contains hallucination (false or invented information) or not. Respond "True" if the answer contains hallucination, and "False" if the answer is fully grounded in the provided context.
        
        Context: {context}
        Answer: {answer}
        
        Output in JSON format:
        hallucination: true or false
    """,
    input_variables=["context", "answer"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
hallu_chain = (
        hallucination_prompt
        | llm
        | JsonOutputParser()
)

if result["relevance"]:
    while True:
        answer = rag_chain.invoke({"context": docs, "question": question})
        hallu_result = hallu_chain.invoke({"context": docs, "answer": answer})
        if not hallu_result["hallucination"]:
            print(answer)
            break
else:
    print("I don't know.")

/Users/hyobinsim/miniconda3/lib/python3.12/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Agent memory refers to a long-term memory module in LLM-powered autonomous agents that records experiences in natural language. It includes observations and events that can inform the agent's behavior through a retrieval model, which prioritizes recency, importance, and relevance. Additionally, a reflection mechanism synthesizes these memories into higher-level inferences, guiding future actions.
